# I was originally doing the predicting itself with LLM but it was taking too long / too expensive.

### RQ: “Does incorporating textbook content features, such as text and images, improve quiz score prediction beyond past student performance on exercises?”

In [ ]:
import pandas as pd

In [ ]:
df_80_validate_features = pd.read_csv("data/filtered80_responses_2023.csv")
df_80_validate_eoc = pd.read_csv("data/filtered80_eoc_2023.csv")

df_unique_pages_abc = pd.read_csv("parse_textbook_content/data/df_pages_content_abc.csv")
df_unique_pages_abcd = pd.read_csv("parse_textbook_content/data/df_pages_content_abcd.csv")

In [ ]:
df_80_validate_features.chapter_num.unique()

In [5]:
df_unique_pages = pd.concat([df_unique_pages_abc, df_unique_pages_abcd], ignore_index=True)

In [6]:
# combine to get all features
book_abc = 'College / Statistics and Data Science (ABC)'
book_abcd = 'College / Advanced Statistics and Data Science (ABCD)'

df_80_validate_features = df_80_validate_features.merge(
    df_unique_pages,
    on=['book', 'page'],
    how='left'
)

In [9]:
# past performance only features
past_performance_cols = ['class_id', 'student_id', 'chapter', 'page', 'item_id', 'item_type', 'points_earned', 'points_possible', 'attempt']

# past performance + text
past_performance_plus_text_cols = ['class_id', 'student_id', 'chapter', 'page', 'item_id', 'item_type', 'points_earned', 'points_possible', 'attempt', 'lrn_response_json', 'attempt', 'page_context_text']

# past performance + images
past_performance_plus_image_cols = ['class_id', 'student_id', 'chapter', 'page', 'item_id', 'item_type', 'points_earned', 'points_possible', 'attempt', 'page_context_images_only']
# why 
# past performance + images + text
past_performance_plus_text_and_image_cols = ['class_id', 'student_id', 'chapter', 'page', 'item_id', 'item_type', 'points_earned', 'points_possible', 'attempt', 'lrn_response_json', 'attempt', 'page_context_text', 'page_context_alt_text_and_images']

# Create Prompts at Chapter Level

In [65]:
from pydantic import BaseModel, Field

class score_prediction(BaseModel):
    explanation: str = Field(..., description="Explanation of your prediction.")
    prediction: float = Field(
        ..., 
        description="Final estimated probability (0-1) a random US adult answers this specific question correctly, integrating all visual, textual, and interaction factors up to 4 decimal places."
    )

USER_PROMPT_END = f"""\nBased on this information, explain your reasoning and predict the probability (between 0 and 1) with 4 decimal places that this student will answer a randomly selected question from the end-of-chapter quiz correctly.

Respond in the following JSON format, conforming to the `score_prediction` schema:

```json
{{
  "explanation": ...
  "prediction": ...
}}"""

In [197]:
import pandas as pd
import numpy as np

# Define system prompts from our prompts.py file
from prompts import (
    system_prompt_past_performance_only,
    system_prompt_past_performance_plus_text,
    system_prompt_past_performance_plus_image,
    system_prompt_past_performance_plus_text_and_image
)

# Helper function to clean text
def clean_text(text):
    if pd.isna(text):
        return ""
    return str(text).strip().replace('\n', ' ').replace('\r', ' ')

# Helper function to create a single exercise entry
def create_chapter_agg_prompt(df_chapter, include_text=False, include_image=False):
    user_prompt = ""
    for _, row in df_chapter.iterrows():
        base = f"\nChapter: {row['chapter']}\nPage Name: {row['page']}\nItem ID: {row['item_id']}\nItem Type: {row['item_type']}\nStudent earned {row['points_earned']} out of {row['points_possible']}."
        
        if include_text and include_image:
            if not pd.isna(row['page_context_text']):
                base += "\nPage Context:\n"
                text = clean_text(row.get('page_context_text', ''))
                if text:
                    base += text

            if not pd.isna(row['page_context_alt_text_and_images']):
                page_context_alt_text_and_images = eval(row.get('page_context_alt_text_and_images', ''))
                for image in page_context_alt_text_and_images:
                    if image[1].startswith('https://'):
                        base += f"\nImage Alt Text: {image[0]}"

        elif include_text :
            if not pd.isna(row['page_context_text']):
                base += "\nPage Context:\n"
                text = clean_text(row.get('page_context_text', ''))
                if text:
                    base += text
        
        elif include_image:
            if not pd.isna(row['page_context_images_only']):
                base += "\nSee images attached\n"

        user_prompt += base
    
    return user_prompt

def create_agg_image_url_list(df_chapter):
    image_url_list = []
    for _, row in df_chapter.iterrows():
        if not pd.isna(row['page_context_images_only']):
            curr_urls = eval(row['page_context_images_only'])
            for url in curr_urls:
                if url.startswith('https://') and (url.endswith('.png') or url.endswith('.jpg') or url.endswith('.jpeg')):
                    image_url_list.append(url)
    return list(set(image_url_list))

def get_agg_list_scores(df_chapter):
    scores = []
    for _, row in df_chapter.iterrows():
        score = row['points_earned'] / row['points_possible']
        scores.append(score)
    return np.mean(scores)

def generate_chapter_prompts_and_image_lists(df):
    grouped = df.groupby(['class_id', 'student_id', 'book', 'chapter_num'])
    
    results = []

    for (class_id, student_id, book, chapter_num), group in grouped:
        # Sort by page for consistency
        group_sorted = group.sort_values(by=['page_num', 'item_id'])

        # Build each of the 4 prompt types using the new helper
        prompt_perf_only = create_chapter_agg_prompt(
            group_sorted, include_text=False, include_image=False
        )

        prompt_plus_text = create_chapter_agg_prompt(
            group_sorted, include_text=True, include_image=False
        )

        prompt_plus_image = create_chapter_agg_prompt(
            group_sorted, include_text=False, include_image=True
        )

        prompt_plus_text_and_image = create_chapter_agg_prompt(
            group_sorted, include_text=True, include_image=True
        )

        image_url_list = str(create_agg_image_url_list(group_sorted))

        scores = np.mean(group_sorted['points_earned'] / group_sorted['points_possible']) # get_agg_list_scores(group_sorted)

        results.append({
            'class_id': class_id,
            'student_id': student_id,
            'book': book,
            'chapter': chapter_num,
            'Prompt_PastPerformanceOnly': prompt_perf_only,
            'Prompt_PastPerformancePlusText': prompt_plus_text,
            'Prompt_PastPerformancePlusImage': prompt_plus_image,
            'Prompt_PastPerformancePlusTextAndImage': prompt_plus_text_and_image,
            'image_url_list': image_url_list,
            'avg_exercise_scores': scores
        })

    return pd.DataFrame(results)


def create_full_prompt_row(row, include_text=False, include_image=False):
    if include_text and include_image:
        system_prompt = system_prompt_past_performance_plus_text_and_image
        user_prompt = row['Prompt_PastPerformancePlusTextAndImage']
    elif include_text:
        system_prompt = system_prompt_past_performance_plus_text
        user_prompt = row['Prompt_PastPerformancePlusText']
    elif include_image:
        system_prompt = system_prompt_past_performance_plus_image
        user_prompt = row['Prompt_PastPerformancePlusImage']
    else:
        system_prompt = system_prompt_past_performance_only
        user_prompt = row['Prompt_PastPerformanceOnly']

    # Add the end of the prompt
    user_prompt += USER_PROMPT_END

    # full prompt with code
    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": [{"type": "text", "text": user_prompt}]}
    ]
    if include_image and type(row['image_url_list']) == str:
        for image_url in eval(row['image_url_list']):
            if image_url.startswith('https://'):
                messages[1]['content'].append({
                    "type": "image_url",
                    "image_url": {
                        "url": image_url,
                    },
                })

    return str(messages)

def create_full_prompt_df(df):
    df['FullPrompt_PastPerformanceOnly'] = df.apply(lambda x: create_full_prompt_row(x, include_text=False, include_image=False), axis=1)
    df['FullPrompt_PastPerformancePlusText'] = df.apply(lambda x: create_full_prompt_row(x, include_text=True, include_image=False), axis=1)
    df['FullPrompt_PastPerformancePlusImage'] = df.apply(lambda x: create_full_prompt_row(x, include_text=False, include_image=True), axis=1)
    df['FullPrompt_PastPerformancePlusTextAndImage'] = df.apply(lambda x: create_full_prompt_row(x, include_text=True, include_image=True), axis=1)
    return df


In [191]:
from openai import OpenAI
import dotenv
import json

dotenv.load_dotenv()
client = OpenAI()

# Function to call GPT-4.1 Nano with image capabilities
def generate_eoc_predictions(row, type, model = "gpt-4.1-nano"):
    messages = eval(row[f'FullPrompt_{type}'])
    response = client.beta.chat.completions.parse(
        model=model,
        messages=messages,
        response_format=score_prediction,
    )
    return response.choices[0].message.content

In [ ]:
# Filter out chapter 2, 3 6
df_80_validate_features_filter = df_80_validate_features[~df_80_validate_features.chapter_num.isin([2, 3, 6])]
df_80_validate_eoc_filter = df_80_validate_eoc[~df_80_validate_eoc.chapter.isin([2, 3, 6])]


df_80_validate_features_filter_sample = df_80_validate_features_filter.groupby(['chapter_num']).apply(lambda x: x.sample(2)).reset_index(drop=True)
df_80_validate_eoc_filter_sample = df_80_validate_eoc_filter.groupby(['chapter']).apply(lambda x: x.sample(2)).reset_index(drop=True)

# sample from each chapter
sample = 2
df_80_validate_features_filter_sample = df_80_validate_features_filter.groupby(['chapter']).apply(lambda x: x.sample(sample)).reset_index(drop=True)


In [145]:
df_80_validate_features_filter_sample = generate_chapter_prompts_and_image_lists(df_80_validate_features_filter_sample)
df_80_validate_features_filter_sample = create_full_prompt_df(df_80_validate_features_filter_sample)

In [146]:
# get predictions for each prompt
df_80_validate_features_filter_sample['prediction_PastPerformanceOnly'] = ""
df_80_validate_features_filter_sample['prediction_PastPerformancePlusText'] = ""
df_80_validate_features_filter_sample['prediction_PastPerformancePlusImage'] = ""
df_80_validate_features_filter_sample['prediction_PastPerformancePlusTextAndImage'] = ""

for i in range(len(df_80_validate_features_filter_sample)):
    df_80_validate_features_filter_sample.loc[i, 'prediction_PastPerformanceOnly'] = generate_eoc_predictions(df_80_validate_features_filter_sample.iloc[i], "PastPerformanceOnly")
    df_80_validate_features_filter_sample.loc[i, 'prediction_PastPerformancePlusText'] = generate_eoc_predictions(df_80_validate_features_filter_sample.iloc[i], "PastPerformancePlusText")
    df_80_validate_features_filter_sample.loc[i, 'prediction_PastPerformancePlusImage'] = generate_eoc_predictions(df_80_validate_features_filter_sample.iloc[i], "PastPerformancePlusImage")
    df_80_validate_features_filter_sample.loc[i, 'prediction_PastPerformancePlusTextAndImage'] = generate_eoc_predictions(df_80_validate_features_filter_sample.iloc[i], "PastPerformancePlusTextAndImage")

In [ ]:
df_80_validate_features_filter_sample.merge(df_80_validate_eoc_filter, on=['class_id', 'student_id', 'book', 'chapter'], how='left')
# Convert prediction columns from string to float
for col in ['prediction_PastPerformanceOnly', 'prediction_PastPerformancePlusText', 
            'prediction_PastPerformancePlusImage', 'prediction_PastPerformancePlusTextAndImage']:
    # Extract the prediction value from the JSON string
    df_80_validate_features_filter_sample[col + "_score"] = df_80_validate_features_filter_sample[col].apply(
        lambda x: float(eval(x)["prediction"]) if isinstance(x, str) else x
    )

# Fix the merge issue by converting chapter types
df_80_validate_features_filter_sample['chapter'] = df_80_validate_features_filter_sample['chapter'].astype(int)
df_80_validate_eoc_filter['chapter'] = df_80_validate_eoc_filter['chapter'].astype(int)

# Merge the dataframes correctly
merged_df = df_80_validate_features_filter_sample.merge(
    df_80_validate_eoc_filter, 
    on=['class_id', 'student_id', 'book', 'chapter'], 
    how='left'
)

# Calculate differences for each prediction type
merged_df['diff_PastPerformanceOnly'] = merged_df['prediction_PastPerformanceOnly_score'] - merged_df['score']
merged_df['diff_PastPerformancePlusText'] = merged_df['prediction_PastPerformancePlusText_score'] - merged_df['score']
merged_df['diff_PastPerformancePlusImage'] = merged_df['prediction_PastPerformancePlusImage_score'] - merged_df['score']
merged_df['diff_PastPerformancePlusTextAndImage'] = merged_df['prediction_PastPerformancePlusTextAndImage_score'] - merged_df['score']

# Calculate average mean difference for each category
avg_diff_past_only = merged_df['diff_PastPerformanceOnly'].abs().mean()
avg_diff_past_text = merged_df['diff_PastPerformancePlusText'].abs().mean()
avg_diff_past_image = merged_df['diff_PastPerformancePlusImage'].abs().mean()
avg_diff_past_text_image = merged_df['diff_PastPerformancePlusTextAndImage'].abs().mean()

print("Average Mean Absolute Differences:")
print(f"Past Performance Only: {avg_diff_past_only:.4f}")
print(f"Past Performance + Text: {avg_diff_past_text:.4f}")
print(f"Past Performance + Image: {avg_diff_past_image:.4f}")
print(f"Past Performance + Text + Image: {avg_diff_past_text_image:.4f}")

# Update the dataframe for further analysis
df_80_validate_features_filter_sample = merged_df


In [ ]:
eval(y)["prediction"]

# Test

In [196]:
df_80_validate_features_added_features = generate_chapter_prompts_and_image_lists(df_80_validate_features)

In [201]:
df_80_validate_features_added_features_and_eoc = df_80_validate_features_added_features.merge(df_80_validate_eoc, on=['class_id', 'student_id', 'book', 'chapter'], how='left')

In [ ]:
df_80_validate_features_added_features_and_eoc

In [231]:
df_80_validate_features_added_features_and_eoc[df_80_validate_features_added_features_and_eoc.index ==4855].to_csv("outputs/df_80_validate_features_added_features_and_eoc_4855.csv", index=False)

In [ ]:
# create 2 columns for number of images and number of words in text
df_80_validate_features_added_features_and_eoc['num_images'] = df_80_validate_features_added_features_and_eoc['page_context_images_only'].apply(lambda x: len(eval(x)) if isinstance(x, str) else 0)
df_80_validate_features_added_features_and_eoc['num_words'] = df_80_validate_features_added_features_and_eoc['page_context_text'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)


df_80_validate_features_added_features_and_eoc.num_words.count()

In [ ]:
# create a regression model to predict the score from the features
from sklearn.linear_model import LinearRegression

# Create a regression model
model = LinearRegression()

# features: avg_exercise_scores
# predict: score

# Filter out rows with missing scores
df_train = df_80_validate_features_added_features_and_eoc.dropna(subset=['score', 'avg_exercise_scores'])

# Prepare features and target
X = df_train[['avg_exercise_scores']]
y = df_train['score']

# Train the model
model.fit(X, y)

# Print model coefficients and score
print("Model coefficients:")
print(f"Intercept: {model.intercept_:.4f}")
print(f"Avg Exercise Scores: {model.coef_[0]:.4f}")
# print(f"Number of Images: {model.coef_[1]:.4f}")
# print(f"Number of Words: {model.coef_[2]:.4f}")

# Evaluate the model
train_score = model.score(X, y)
print(f"R² score on training data: {train_score:.4f}")

# Make predictions on training data
y_pred = model.predict(X)

# Calculate mean absolute error
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(y, y_pred)
print(f"Mean Absolute Error: {mae:.4f}")


In [ ]:
# get average score over all the chapters
df_80_validate_features_added_features_and_eoc.score.mean()



In [222]:
# Filter df_20_test_features to only include rows where the combination of student_id, class_id, and chapter exists in df_20_test_eoc
df_20_test_features_removed_rows = df_20_test_features.merge(
    df_20_test_eoc[['student_id', 'class_id', 'chapter']],
    on=['student_id', 'class_id', 'chapter'],
    how='right'
)


In [224]:
# use that model to predict the score for the test set
x_test = df_20_test_features_removed_rows[['avg_exercise_scores']].fillna(model.coef_[0]) # fillna with the average score over all the chapters

y_pred_test = model.predict(x_test)

In [227]:
# Convert predictions to a DataFrame with id and score columns
predictions_df = pd.DataFrame({
    'id': range(len(y_pred_test)),
    'score': y_pred_test
})

# Display the DataFrame
predictions_df.to_csv("outputs/df_20_test_features_predictions.csv", index=False)

In [ ]:
df_20_test_features_raw = pd.read_csv("data/filtered20_responses_2023.csv")
df_20_test_eoc = pd.read_csv("data/filtered20_eoc_2023.csv")

In [193]:
# Merge EOC data with page content for ABC textbook
df_20_test_features_raw = df_20_test_features_raw.merge(
    df_unique_pages,
    on=['book', 'page'],
    how='left'
)

df_20_test_features = generate_chapter_prompts_and_image_lists(df_20_test_features_raw)
# df_20_test_features = create_full_prompt_df(df_20_test_features)

In [ ]:
# get predictions for each prompt
df_20_test_features['prediction_PastPerformanceOnly'] = ""
df_20_test_features['prediction_PastPerformancePlusText'] = ""
df_20_test_features['prediction_PastPerformancePlusImage'] = ""
df_20_test_features['prediction_PastPerformancePlusTextAndImage'] = ""

print("Each df has rows: ", len(df_20_test_features))
# print("Total API calls needed: ", len(df_20_test_features) * 4)
for i in range(len(df_20_test_features)):
    if i % 100 == 0:
        print(f"On API call: {i} out of {len(df_20_test_features) * 4}")
    df_20_test_features.loc[i, 'prediction_PastPerformanceOnly'] = generate_eoc_predictions(df_20_test_features.iloc[i], "PastPerformanceOnly")
    df_20_test_features.loc[i, 'prediction_PastPerformancePlusText'] = generate_eoc_predictions(df_20_test_features.iloc[i], "PastPerformancePlusText")
    df_20_test_features.loc[i, 'prediction_PastPerformancePlusImage'] = generate_eoc_predictions(df_20_test_features.iloc[i], "PastPerformancePlusImage")
    df_20_test_features.loc[i, 'prediction_PastPerformancePlusTextAndImage'] = generate_eoc_predictions(df_20_test_features.iloc[i], "PastPerformancePlusTextAndImage")


In [159]:
df_20_test_features.to_csv("outputs/df_20_test_features_predictions.csv", index=False)

In [ ]:
# create 2 columns for number of images and number of words in text
df_20_test_features['num_images'] = df_20_test_features['page_context_images_only'].apply(lambda x: len(eval(x)) if isinstance(x, str) else 0)
df_20_test_features['num_words'] = df_20_test_features['page_context_text'].apply(lambda x: len(x.split()) if isinstance(x, str) else 0)


In [169]:
df_x = pd.read_csv("outputs/df_20_test_features_predictions.csv")